In [1]:
%pip install tqdm logging python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Using cached logging-0.4.9.6.tar.gz (96 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 14, in <module>
        File "C:\Users\oheit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\setuptools\__init__.py", line 21, in <module>
          import _distutils_hack.override  # noqa: F401
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\oheit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\_distutils_hack\override.py", line 1, in <module>
          __import__('_distutils_hack').do_override()
        File "C:\Users\oheit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\_distu

In [2]:
import logging
import time
from tqdm import tqdm
from typing import Optional
from dotenv import load_dotenv
import json

In [3]:
from os import path, getenv
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from client.openUiClient import OpenUiClient

Added ai to sys.path


In [4]:
client = OpenUiClient()

In [5]:
models = [
    # 'deepseek-r1:14b', 
          'llama3:8b', 
          'qwen:1.8b', 
          'phi:latest', 
          'stable-code:latest', 
          #'gemma3:12b', 
          # 'phi4-reasoning:latest',
          #'llama3.2-vision:11b'
          ]

In [6]:
def read_json(file_path):
        """
        Read a JSON file and return its content.
        
        :param file_path: The path of the JSON file to read.
        :return: The content of the JSON file as a dictionary.
        """
        try:
            file_path
            with open(file_path, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            logging.error(f"File not found: {file_path}")
            return None
        except json.JSONDecodeError as e:
            logging.error(f"Error decoding JSON from {file_path}: {e}")
            return None

In [7]:
def test_chatWithSeveralModels():
    file_path = path.abspath(path.join('..', '..', 'golden_set.json'))
    json_data = read_json(file_path)
    responses = []
    
    knowledge_file = path.abspath(path.join('..','knowledge', 'Guidelines.txt'))
    
    for index, item in json_data.items():
        repos = item
        for repo in tqdm(repos, desc='Processing repos', leave=False):
            main_content = []
            commits = repo.get('commits', [])

            for commit in tqdm(commits, desc='Processing commits', leave=False):
                commit_message = commit.get('message', '')
                
                files = commit.get('files', [])
                commit_message = commit.get('message', '')
                link = commit.get('link', '')
                sha = commit.get('sha', '')
                files = commit.get('files', [])
                main_content.append((link, f"""Commit Message: {commit_message}
                                    Files in the commit:
                                    {files}""",
                                    sha))
            
            for content in main_content:
                for model in models:
                    print(f"\nUsing model: {model}")
                    max_retries = 1
                    for attempt in range(max_retries):
                        chat_response = client.chatWithModel(knowledge=knowledge_file, commit_data=content[1], model=model)
                        if chat_response:
                            print("Status code:", chat_response.status_code)
                            try:
                                chat_response_message = chat_response.json()['choices'][0]['message']['content']
                                print("Chat response:", chat_response_message)
                                responses.append({
                                    'model': model,
                                    'sha': content[2],
                                    'link': content[0],
                                    'response': chat_response_message,
                                })
                                break
                            except (KeyError, IndexError) as e:
                                print(f"Error processing the chat response. Details: {e}")
                                break
                        else:
                            print("Unable to perform the chat.")
                            if attempt < max_retries - 1:
                                print(f"Retrying ({attempt+1}/{max_retries}) after 10 seconds...")
                                time.sleep(5)
                            else:
                                print("Max retries reached. Skipping this request.")
                    time.sleep(10)
    return responses

In [8]:
def test_chatWithModel(model: Optional[str] = 'llama3.1:8b'):
    filePath = path.abspath(path.join('..', '..', 'golden_set.json'))
    json_data = read_json(filePath)
    responses = []
    
    knowledge_file = path.abspath(path.join('..','knowledge', 'Guidelines.txt'))

    for index, item in json_data.items():
        repos = item
        for repo in tqdm(repos, desc='Processing repos', leave=False):
            main_content = []
            commits = repo.get('commits', [])

            for commit in tqdm(commits, desc='Processing commits', leave=False):
                commit_message = commit.get('message', '')
                link = commit.get('link', '')
                sha = commit.get('sha', '')
                files = commit.get('files', [])
                main_content.append((link, f"""Commit Message: {commit_message}
                                    Files in the commit:
                                    {files}""",
                                    sha))
            
            for content in main_content:
                chat_response = client.chatWithModel(knowledge=knowledge_file, commit_data=content[1], model=model)
                if chat_response:
                    print("Status code:", chat_response.status_code)
                    try:
                        chat_response_message = chat_response.json()['choices'][0]['message']['content']
                        print("Chat response:\n", chat_response_message)
                        responses.append({
                            'model': model,
                            'sha': content[2],
                            'link': content[0],
                            'response': chat_response_message,
                        })
                    except (KeyError, IndexError) as e:
                        print(f"Error processing the chat response. Details: {e}")
                else:
                    print("Unable to perform the chat.")
                time.sleep(0.5)
    return responses

In [ ]:
def save_output(output: str, file_path: str):
    """
    Save the output of model(s) to a json file.
    
    The output will be restructured to a dictionary format with model as the key and the sha, link and response as values.
    
    :param output: The content to save.
    """
    output_dict = {}
    for response in output:
        model = response['model']
        if model not in output_dict:
            output_dict[model] = []
        output_dict[model].append({
            'sha': response['sha'],
            'link': response['link'],
            'response': response['response']
        })
    
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(output_dict, file, indent=4)
    print(f"Output saved to {file_path}")

In [10]:
print(client.getAvailableApiModels())

['deepseek-r1:14b', 'llama3:8b', 'llama3.1:8b', 'llama3.2-vision:11b', 'medllama2:7b', 'phi:latest', 'qwen:0.5b', 'qwen:1.8b', 'qwen:4b', 'stable-code:3b', 'stable-code:latest', 'gemma3:12b', 'gemma3:1b', 'gemma3:4b', 'phi4:14b', 'qwen2.5:14b', 'phi4-reasoning:latest']


In [ ]:
output_stable_code = test_chatWithModel("llama3:8b")
save_output(output_stable_code, path.abspath(path.join('..','..','output_for_llama3_model.txt')))

Processing repos:   0%|          | 0/4 [00:00<?, ?it/s]

Error performing the chat. Details: 504 Server Error: Gateway Time-out for url: https://llm.ic.unicamp.br/api/chat/completions
Unable to perform the chat.


In [ ]:
# output_all_models = test_chatWithSeveralModels()
# save_output(output_all_models, path.abspath(path.join('..','..','output_for_all_models.txt')))

Processing repos:   0%|          | 0/4 [00:00<?, ?it/s]


Using model: llama3:8b
Error performing the chat. Details: 504 Server Error: Gateway Time-out for url: https://llm.ic.unicamp.br/api/chat/completions
Unable to perform the chat.
Max retries reached. Skipping this request.

Using model: qwen:1.8b
Error performing the chat. Details: 504 Server Error: Gateway Time-out for url: https://llm.ic.unicamp.br/api/chat/completions
Unable to perform the chat.
Max retries reached. Skipping this request.

Using model: phi:latest
Error performing the chat. Details: 504 Server Error: Gateway Time-out for url: https://llm.ic.unicamp.br/api/chat/completions
Unable to perform the chat.
Max retries reached. Skipping this request.

Using model: stable-code:latest
Error performing the chat. Details: 504 Server Error: Gateway Time-out for url: https://llm.ic.unicamp.br/api/chat/completions
Unable to perform the chat.
Max retries reached. Skipping this request.

Using model: llama3:8b
Error performing the chat. Details: 504 Server Error: Gateway Time-out for